In [21]:
import pandas as pd 
from scipy.stats import *
import scipy.stats as stats
import glob

In [12]:
# Generate simulated examples for the unannotated model
# Let's do 20 replicates of N = 1000

for i in range(20):
    gamma_n = []
    Z_n = []
    num_severe_n = []
    freq_n = []
    num_freq_n = []
    eta_n = []
    
    eta = beta.rvs(1, 100)
    
    for n in range(1000):
        eta_n.append(eta)
        
        gamma = stats.gamma.rvs(2, 1)
        gamma_n.append(gamma)
        
        Z = bernoulli.rvs(eta)
        Z_n.append(Z)
        
        num_freq = poisson.rvs(32) + 1
        num_freq_n.append(num_freq)
        freq = num_freq / 1000
        freq_n.append(freq)
        
        if Z == 0:
            num_severe = binom.rvs(num_freq, freq)
        else:
            num_severe = binom.rvs(num_freq, freq * gamma)
        num_severe_n.append(num_severe)
        
    df = pd.DataFrame({
        'variants': np.arange(0, 1000, 1),
        'num_severe': num_severe_n,
        'frequency': freq_n,
        'num_freq': num_freq_n,
        'gamma': gamma_n,
        'Z': Z_n,
        'eta': eta_n
    })
    
    df.to_csv('simulated_datasets/unannotated/{0}_sim.csv'.format(i), index=False)

In [24]:
all_results = glob.glob('simulated_datasets/unannotated/results/all/*/summaries/gamma_i-summary.csv')

In [28]:
a = pd.read_csv('simulated_datasets/unannotated/{0}_sim.csv'.format(0))
b = pd.read_csv('simulated_datasets/unannotated/results/all/rep ({0}).exec/summaries/gamma_i-summary.csv'.format(1))

In [30]:
b

,Unnamed: 0,variants,mean,sd,min,median,max,HDI.lower,HDI.upper
0,1,0,1.952856,1.417338,0.091099,1.561589,6.608434,0.091099,3.938032
1,2,1,1.944575,1.303759,0.051943,1.637452,5.853276,0.051943,3.526581
2,3,2,1.947038,1.235622,0.117848,1.625900,6.056115,0.513418,3.761553
3,4,3,2.192779,1.469247,0.210647,2.165445,7.807462,0.210647,4.317167
4,5,4,1.996711,1.438873,0.071259,1.647286,6.741945,0.195835,4.005501
...,...,...,...,...,...,...,...,...,...
995,996,995,2.206892,1.673332,0.096258,1.948614,8.475231,0.096258,4.512951
996,997,996,1.841323,1.283188,0.092023,1.506069,6.625294,0.140851,3.661506
997,998,997,1.982506,1.368035,0.103510,1.647744,6.807940,0.103510,3.901806
998,999,998,2.014210,1.574851,0.143619,1.554446,8.568976,0.143619,3.879624


In [31]:
a

,variants,num_severe,frequency,num_freq,gamma,Z,eta
0,0,0,0.026,26,5.890023,0,0.003069
1,1,0,0.031,31,2.132569,0,0.003069
2,2,1,0.035,35,3.683523,0,0.003069
3,3,0,0.024,24,1.547232,0,0.003069
4,4,3,0.042,42,3.520144,0,0.003069
...,...,...,...,...,...,...,...
995,995,1,0.041,41,1.349794,0,0.003069
996,996,0,0.033,33,2.816755,0,0.003069
997,997,1,0.038,38,1.543329,0,0.003069
998,998,2,0.034,34,1.827931,0,0.003069


In [38]:
positive_hits = 0

for i in range(20):
    data_df = pd.read_csv('simulated_datasets/unannotated/{0}_sim.csv'.format(i))
    results_df = pd.read_csv('simulated_datasets/unannotated/results/all/rep ({0}).exec/summaries/gamma_i-summary.csv'.format(i+1))
    
    for i,row in data_df.iterrows():
        true_gamma = row['gamma']
        variant = row['variants']
        
        results_row = results_df.loc[results_df['variants'] == variant]
        HDI_low = results_row['HDI.lower'].values[0]
        HDI_high = results_row['HDI.upper'].values[0]
        
        if HDI_low <= true_gamma <= HDI_high:
            positive_hits += 1
            
print(positive_hits / (20 * 1000))

0.7742
